# Taproot BIP 322 Signing

This notebook experiments with using taproot based addresses for BIP 322 signing

## Types of Taproot Scripts to Test

See Jimmy Song presentation on [Taproot MultiSig](https://jimmysong.github.io/taproot-multisig/)

1. Schnorr pay to public key
2. Schnorr MuSig signer (n of n)
3. Single leaf multi sig TapScript
4. Multi leaf k-of-k TapScript
5. Multi leaf MuSig TapScript
6. Multi leaf everything TapScript

Initially, this notebook will just focus creating and verify BIP 322 signatures using types 1 and 2. Although the presentation referenced above details why the other types may be interesting to support.

In [ ]:
cd ..

## Schnorr pay to public key

This just uses a taproot based address which can be unlocked with a single schnorr signature from a specific private key.

### Import PrivateKey

In [ ]:
from buidl.ecc import PrivateKey

In [ ]:
private_key_wif = "L3VFeEujGtevx9w18HD1fhRbCH67Az2dpCymeRE1SoPK6XQtaN2k"

In [ ]:
private_key = PrivateKey.parse(private_key_wif)

### Define Message to Sign

In [ ]:
message = "Hello World"

### Generate A Pay to Tap Root Script for the Secp256k1 Point associated with The Private Key

In [ ]:
p2tr_script = private_key.point.p2tr_script()
print(p2tr_script)

### Convert Script to Bitcoin Address Representation

In [ ]:
p2tr_address = p2tr_script.address()
print(p2tr_address)

### Calculate Teaked Private Key for the P2TR Address

See the Tweaked Keys section :  https://medium.com/coinmonks/on-bitcoins-schnorr-signature-algorithm-and-taproot-script-and-witness-sizes-fe4d1e6591a7

In [ ]:
from buidl.taproot import TapRoot
taproot = TapRoot(private_key.point)
tweaked_private_key = private_key.tweaked(taproot.tweak)

In [ ]:
taproot

### Sign Message using Tweaked Private Key and Taproot Address

In [ ]:
from src.message import sign_message, MessageSignatureFormat
signature = sign_message(MessageSignatureFormat.SIMPLE, tweaked_private_key, p2tr_address, message)
print("BIP 322 p2tr based Signature: ", signature)

### Verify Signature against p2tr_address and message

In [ ]:
from src.message import verify_message

is_valid = verify_message(p2tr_address, signature, message)

print("BIP 322 Signature \n" + signature + "\nagainst message : " + message + " and address : " + p2tr_address + "\nis valid? ", is_valid)

In [ ]:
is_valid

## Schnorr MuSig Signing

This is a p2tr address that requires a n-of-n Schnorr aggregated signature.

**Note: The current API defined in message.py cannot support MuSig. Or other more complex Bitcoin Script. Need something along the lines of Partially Signed BIP 322 Signature**

**Note2: PSBT in buidl-python does not to support taproot yet (See open issue https://github.com/buidl-bitcoin/buidl-python/issues/107)**

In [ ]:
secret1 = 123817492183
priv_key1 = PrivateKey(secret=secret1)

secret2 = 7529884329432
priv_key2 = PrivateKey(secret=secret2)

In [ ]:
aggregated_pubkey = priv_key1.point + priv_key2.point

In [ ]:
taproot = TapRoot(aggregated_pubkey)
taproot.tweak

In [ ]:
musig_taproot_address = taproot.address()

In [ ]:
tweaked_pk_1 = priv_key1.tweaked(taproot.tweak)

In [ ]:
signature = sign_message(MessageSignatureFormat.FULL,tweaked_pk_1,musig_taproot_address,message)